# Albmentations

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import albumentations as albu
import cv2
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt 
import pandas as pd 

from lane_detection_hackathon.baseparser import BaseParser
from lane_detection_hackathon.utils import fs

CURRENT_DPATH = os.path.abspath(os.path.dirname("__file__"))
PROJECT_ROOT = os.path.abspath(os.path.join(CURRENT_DPATH, os.pardir, os.pardir))
DATA_DPATH = os.path.join(PROJECT_ROOT, "data")

## Data Loading

In [ ]:
fpath = os.path.join(DATA_DPATH, "datasets", "apolloscape", "2023_02_24", "raw_data.xlsx")
full_df = pd.read_excel(fpath, index_col=0)

full_df.shape

## Albumentations

### ShiftScaleRotate

In [ ]:
transform = albu.ShiftScaleRotate(
    shift_limit=0.1,
    scale_limit=0.1,
    rotate_limit=45,
    interpolation=3,
    border_mode=cv2.BORDER_CONSTANT,
    p=1.0,
    value=(0, 0, 0),
)

## Vizualization

In [ ]:
@interact
def show(index=IntSlider(value=0, min=0, max=full_df.shape[0]-1)):
  row = full_df.iloc[index]

  src_fpath = row[BaseParser.src_key]
  mask_fpath = row[BaseParser.target_key]

  full_src_fpath = os.path.join(DATA_DPATH, src_fpath)
  full_mask_fapth = os.path.join(DATA_DPATH, mask_fpath)

  src_image = fs.read_image(full_src_fpath)
  trm_image = transform(image=src_image)["image"]

  mask_image = fs.read_image(full_mask_fapth)
  trm_mask = transform(image=mask_image)["image"]

  fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))

  ax[0][0].imshow(src_image)
  ax[0][0].set_title("Src Image")

  ax[0][1].imshow(trm_image)
  ax[0][1].set_title(f"SRC: {transform.__class__.__name__}")

  ax[1][0].imshow(mask_image)
  ax[1][0].set_title("Mask")

  ax[1][1].imshow(trm_mask)
  ax[1][1].set_title(f"MASK: {transform.__class__.__name__}")

  plt.show()